In [3]:
from dotenv import load_dotenv
load_dotenv()

from huggingface_hub import login
import os
login(token=os.getenv("HUGGINGFACE_API_KEY"))

In [4]:
import torch
print(torch.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

2.8.0+cu126
cuda


In [ ]:
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch
from drugocr import extract_text

model_id = "google/gemma-3-1b-it"

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = Gemma3ForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [17]:
image_path = "C:\Potenup\Drug-Detection-Chatbot\data\medicine_00806.jpeg"
ocr_texts = extract_text(image_path)
ocr_texts_join = ','.join(ocr_texts)
print(ocr_texts_join)

[원료약품 및 분량] 이 약 1캡슐 중,유효성분:두타스테리드(USP)0.5mg,첨가제(동물유래성분):젤라틴(기원동물:소,사용,부위:가죽),기타첨가제:농글리세린,부틸히드록시톨루엔,숙,신산젤라틴,폴리옥실40경화피마자유,프로필렌글,리콜모노라우레이트,폴록사머124,D-소르비톨액,[성상],무색투명한 내용물이 들어있는 미황색의 투명한,타원형 연질캡슐,[효능·효과],양성 전립선 비대증 증상의 개선,급성 성 요저류 위험성,감소,양성 전립선 비대증과 관련된 수술 필요성 감소,,성인 남성(만18~50세)의 남성형 탈모(androgenetic,alopecia)의 치료,[용법·용량],이 약의 권장용량은 1일 1회 1캡슐(0.5mg)이다.,캡슐 내용물에 노출시 구강 인두점막의 자극을 초래,할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로,삼켜 복용해야 한다. 이 약은 식사와 관계없이 복용할,수 있다.신장애 환자 또는 노인 환자에서 이 약의,용량을 조절할 필요는 없다. 간장애 환자에게 이 약을,투여한 자료가 없기 때문에 간장애 환자에서의 이,약의 권장용량은 확립되어 있지 않다.,[사용상의 주의사항],1.경고1)여성에게 노출시 남자 태아에 미치는 위험성 이 약은,피부를 통 통해서 서 흡수된다.따라서 이 약의 흡수 가능성과 남자,태아에게 미치는 태자 기형의 위험 가능성 때문에 임신했거나,임신기 가능성이 있는 여성이 이 약을 취급해서는 안든 된다. 또,,여성은 이 약을 취급할 때마다 주의해야 하고 누출되는 캡슐,과의 접촉을 피해야 한다. 만약 캡슐이 새어 이 약과 접촉한,경우에는 접촉부위를 즉시 물과 비누로 세척해야 한다.,이하 첨부문서 참조,[저장방법] 밀폐용기,30°C이하 보관,[제조의뢰자](주)제뉴원사이언스,세종특별자치시 I|전의면 산단길 245,[제조자](주)유유제약,충북 제천시 바이오밸리 1로 94


In [20]:
system_prompt = """
당신은 보수적인 약학 전문가입니다. 약학 전문가로서, 
사용자가 제공한 OCR 텍스트 데이터를 확인해서 정확한 JSON 데이터를 만들어서 제공하는게 중요합니다. 


규칙:
- 제공된 데이터에 없는 항목은 **반드시 "없음"이라고 작성**할 것. 
- 예측이나 추론을 하지 말고, 텍스트를 줄이지 말고 **원본 텍스트를 그대로 작성**할 것. 
- **제품명**: 약의 **제품명**은 별도로 표기된 경우에만 추출하며, '원료약품 및 분량', '성분/함량' 등 **원료나 성분을 설명하는 해당 아래에 있는 텍스트는 제품명으로 간주하지 않습니다**. 
- 제품명이 없으면 "없음"으로 작성하십시오. 
- 성분/함량 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 첨가물/첨가제 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 출력은 반드시 JSON 형식만 출력하고, 그 외 텍스트(설명, 문장)는 절대 포함하지 말 것. 
- **효능/효과**: '효능' 또는 '효과'로 명시된 모든 내용을 **하나도 빠짐없이** 모든 리스트를 작성하십시오.
- OCR 줄바꿈이나 잘림으로 인해 분리된 단어는 반드시 이어 붙여 원래 단어 그대로 복원할 것.
- JSON의 각 key는 반드시 아래 구조를 따를 것

[출력형식]  
{{ 
    '제품명' : "제품의 이름, 없으면 없다고 작성할 것", 
    '성분/함량' : "제품의 각 성분, 유효성분, 함량에 대한 튜플(성분명, 함량)들의 리스트, 없으면 없다고 작성할 것" 
    '첨가물' : "제품의 첨가물/첨가제들의 리스트, 없으면 없다고 작성할 것", 
    '제형/성상' : "제형, 성상, 없으면 없다고 작성할 것", 
    'KPIC/ATC' : 'KPIC/ATC에 대한 리스트, 없으면 없다고 작성할 것", 
    '구분' : "구분, 없으면 없다고 작성할 것" 
    '효능' : "여러가지 효능/효과에 대한 모든 리스트, 없으면 없다고 작성할 것", 
    '용법' : "여러가지 용법에 대한 리스트, 없으면 없다고 작성할 것", 
    '주의사항' : "주의사항 리스트, 없으면 없다고 작성할 것" 
}}
"""

messages = [
    [
        {
            "role":"system",
            "content": [{"type": "text", "text": system_prompt},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": ocr_texts_join}]
        }
    ]
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = True,
    return_dict = True,
    return_tensors="pt"
).to(model.device).to(torch.bfloat16)


with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=1024)

outputs = tokenizer.batch_decode(outputs)
print(outputs)

Attempting to cast a BatchEncoding to type torch.bfloat16. This is not supported.


['<bos><start_of_turn>user\n\n당신은 보수적인 약학 전문가입니다. 약학 전문가로서, \n사용자가 제공한 OCR 텍스트 데이터를 확인해서 정확한 JSON 데이터를 만들어서 제공하는게 중요합니다. \n\n\n규칙:\n- 제공된 데이터에 없는 항목은 **반드시 "없음"이라고 작성**할 것. \n- 예측이나 추론을 하지 말고, 텍스트를 줄이지 말고 **원본 텍스트를 그대로 작성**할 것. \n- **제품명**: 약의 **제품명**은 별도로 표기된 경우에만 추출하며, \'원료약품 및 분량\', \'성분/함량\' 등 **원료나 성분을 설명하는 해당 아래에 있는 텍스트는 제품명으로 간주하지 않습니다**. \n- 제품명이 없으면 "없음"으로 작성하십시오. \n- 성분/함량 \',\' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. \n- 첨가물/첨가제 \',\' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. \n- 출력은 반드시 JSON 형식만 출력하고, 그 외 텍스트(설명, 문장)는 절대 포함하지 말 것. \n- **효능/효과**: \'효능\' 또는 \'효과\'로 명시된 모든 내용을 **하나도 빠짐없이** 모든 리스트를 작성하십시오.\n- OCR 줄바꿈이나 잘림으로 인해 분리된 단어는 반드시 이어 붙여 원래 단어 그대로 복원할 것.\n- JSON의 각 key는 반드시 아래 구조를 따를 것\n\n[출력형식]  \n{{ \n    \'제품명\' : "제품의 이름, 없으면 없다고 작성할 것", \n    \'성분/함량\' : "제품의 각 성분, 유효성분, 함량에 대한 튜플(성분명, 함량)들의 리스트, 없으면 없다고 작성할 것" \n    \'첨가물\' : "제품의 첨가물/첨가제들의 리스트, 없으면 없다고 작성할 것", \n    \'제형/성상\' : "제형, 성상, 없으면 없다고 작성할 것", \n

In [21]:
print(outputs[0].split("<start_of_turn>model")[-1])


```json
{
    "제품명": "두타스테리드",
    "성분/함량": "유효성분: 두타스테리드(USP) 0.5mg, 첨가제(동물유래성분): 젤라틴(기원동물: 소, 사용, 부위: 가죽), 기타 첨가제: 농글리세린, 숙, 신산젤라틴, 폴리옥실40경화피마자유, 프로필렌글, 리콜모노라우레이트, 폴록사머124, D-소르비톨액",
    "첨가물": "동물유래성분",
    "제형/성상": "투명한 내용물이 들어있는 미황색의 투명한, 타원형 연질캡슐",
    "KPIC/ATC": "없음",
    "구분": "없음",
    "효능": [
        "양성 전립선 비대증 증상의 개선, 급성 성 요저류 위험성, 감소, 성인 남성(만 18~50세)의 남성형 탈모(androgenetic, alopecia)의 치료",
        "성인 남성(만 18~50세)의 남성형 탈모(androgenetic, alopecia)의 치료"
    ],
    "용법": "이 약의 권장 용량은 1일 1회 1캡슐(0.5mg)이다. 캡슐 내용물에 노출시 구강 인두점막의 자극을 초래, 할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로 삼켜 복용해야 한다. 이 약은 식사와 관계없이 복용할 수 있다. 신장애 환자 또는 노인 환자에서 이 약의 용량을 조절할 필요는 없다. 간장애 환자에게 이 약의 용량을 투여 자료가 없기 때문에 간장애 환자에서의 이 약의 권장 용량은 확립되어 있지 않다.",
    "주의사항": [
        "여성에게 노출시 남자 태아에 미치는 위험성",
        "이 약의 흡수 가능성과 남자 태아에게 미치는 태자 기형의 위험 가능성 때문에 임신했거나, 임신했거나, 임신 중인 여성에게는 이 약을 취급해서는 안 된다. 또, 여성은 이 약을 취급할 때마다 주의해야 하고 누출되는 캡슐과의 접촉을 피해야 한다.",
        "캡슐이 새어 이 약과 접촉한 경우, 접촉부위를 즉시 물과 비누로 세척해야 한다."
    ]
}
```<end_of_turn>
